# Week 1, Class 2: Introduction to Machine Learning in Healthcare
## Hands-on Lab: PyTorch Fundamentals & Your First Neural Network

**Course:** AI/ML in Medicine and Healthcare  
**Module:** Week 1 - Foundations  

---

## 📝 Student Information

**Student 1:**
- Name: Rodrigo Paoliello de Medeiros
- Email: rodrigopm6@al.insper.edu.br

**Student 2:**
- Name: Gabriel Hermida Mendonça
- Email: gabrielmmh@al.insper.edu.br

**Date Submitted:** 12/02/2026

---

## 🎯 Learning Objectives

By the end of this lab, you will:
1. ✅ Master essential PyTorch tensor operations
2. ✅ **Build a complete neural network from scratch**
3. ✅ **Implement the full training loop** (forward, loss, backward, optimize)
4. ✅ **Evaluate with medical metrics** (sensitivity, specificity)
5. ✅ **Experiment with architectures** and hyperparameters

---

## ⏱️ Time Allocation

- **Part 1:** Tensor Operations (30 minutes)
- **Part 2:** Neural Network (60 minutes)
  - Milestones 1-5: ~50 minutes
  - Milestone 6 (Experimentation): ~30+ minutes

**Total:** 90 minutes


---

# 🚀 PART 1: Tensor Operations Speed Run (30 minutes)

## Instructions:

Complete the following challenges. If you're **experienced with PyTorch**, this should take 15-20 minutes. If you're **new to PyTorch**, take your time and ask questions!

**Format:** Each challenge is independent. Complete as many as you can.

---

In [1]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu128
CUDA available: True


## Challenge 1: Create Medical Data Tensors (5 minutes)

**Task:** Create a tensor representing **100 patients** with **8 medical features** each.

Features:
1. Age (years): 20-80
2. BMI (kg/m²): 18-40
3. Blood Pressure (mmHg): 80-180
4. Glucose (mg/dL): 70-200
5. Insulin (μU/mL): 0-200
6. Pregnancies: 0-15
7. Skin Thickness (mm): 0-99
8. Diabetes Pedigree Function: 0.0-2.5

**Requirements:**
- Use realistic value ranges for each feature
- Final tensor shape: `(100, 8)`
- Data type: `float32`

In [5]:
# TODO: Create a tensor with 100 patients and 8 features
# Hint: You can use torch.randn() and scale/shift to appropriate ranges
# Or create individual feature columns and concatenate

num_patients = 100

patients_data = torch.cat([
    torch.clamp(torch.randn(num_patients, 1) * 15 + 50, min=20, max=80),    # Age: mean=50, std=15 (approx 95% in 20-80)
    torch.clamp(torch.randn(num_patients, 1) * 5.5 + 29, min=18, max=40),    # BMI: mean=29, std=5.5 (approx 95% in 18-40)
    torch.clamp(torch.randn(num_patients, 1) * 25 + 130, min=80, max=180),   # Blood Pressure: mean=130, std=25
    torch.clamp(torch.randn(num_patients, 1) * 32.5 + 135, min=70, max=200), # Glucose: mean=135, std=32.5
    torch.clamp(torch.randn(num_patients, 1) * 50 + 100, min=0, max=200),    # Insulin: mean=100, std=50
    torch.clamp(torch.randn(num_patients, 1) * 3.75 + 7.5, min=0, max=15).round(), # Pregnancies: mean=7.5, std=3.75 (integer)
    torch.clamp(torch.randn(num_patients, 1) * 24.75 + 49.5, min=0, max=99), # Skin Thickness: mean=49.5, std=24.75
    torch.clamp(torch.randn(num_patients, 1) * 0.625 + 1.25, min=0.0, max=2.5) # Diabetes Pedigree Function: mean=1.25, std=0.625
], dim=1).to(torch.float32)



# Verification (don't modify)
assert patients_data is not None, "You need to create the patients_data tensor!"
assert patients_data.shape == (100, 8), f"Expected shape (100, 8), got {patients_data.shape}"
assert patients_data.dtype == torch.float32, f"Expected dtype float32, got {patients_data.dtype}"
print("✓ Challenge 1 complete!")
print(f"Tensor shape: {patients_data.shape}")
print(f"Sample patient data:\n{patients_data[0]}")

✓ Challenge 1 complete!
Tensor shape: torch.Size([100, 8])
Sample patient data:
tensor([ 28.9723,  28.3285, 138.1012,  93.5830,  51.8733,  14.0000,  17.9821,
          1.6716])


## Challenge 2: One-Hot Encoding (5 minutes)

**Task:** Convert diagnosis labels to one-hot encoded vectors.

Given: `labels = [0, 1, 1, 0, 1]` (0 = no diabetes, 1 = diabetes)

**Expected output:**
```
[[1, 0],  # Patient 0: no diabetes
 [0, 1],  # Patient 1: diabetes
 [0, 1],  # Patient 2: diabetes
 [1, 0],  # Patient 3: no diabetes
 [0, 1]]  # Patient 4: diabetes
```

**Hint:** Use `torch.nn.functional.one_hot()`

In [9]:
import torch.nn.functional as F

# Given labels
labels = torch.tensor([0, 1, 1, 0, 1])


one_hot_labels = F.one_hot(labels, num_classes=2)


# Verification (don't modify)
assert one_hot_labels is not None, "You need to create one_hot_labels!"
assert one_hot_labels.shape == (5, 2), f"Expected shape (5, 2), got {one_hot_labels.shape}"
assert torch.equal(one_hot_labels[0], torch.tensor([1, 0])), "First patient encoding incorrect"
print("✓ Challenge 2 complete!")
print(f"One-hot encoded labels:\n{one_hot_labels}")

✓ Challenge 2 complete!
One-hot encoded labels:
tensor([[1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1]])


## Challenge 3: Feature Normalization (5 minutes)

**Task:** Normalize the patient data to have **mean=0** and **standard deviation=1** for each feature.

**Formula:** `normalized = (x - mean) / std`

**Requirements:**
- Normalize each of the 8 features independently
- Use the `patients_data` tensor from Challenge 1
- Verify: mean ≈ 0, std ≈ 1 for each feature

In [10]:
# TODO: Normalize patients_data
# Hint: check numpy or pandas API for mean and stdev functions

# Calculate mean and standard deviation for each feature
mean = patients_data.mean(dim=0)
std = patients_data.std(dim=0)

# Apply normalization
normalized_data = (patients_data - mean) / std

# Verification (don't modify)
assert normalized_data is not None, "You need to create normalized_data!"
assert normalized_data.shape == patients_data.shape, "Shape should not change"

# Check normalization
means = normalized_data.mean(dim=0)
stds = normalized_data.std(dim=0)
print("✓ Challenge 3 complete!")
print(f"Mean per feature (should be ~0): {means}")
print(f"Std per feature (should be ~1): {stds}")
assert torch.allclose(means, torch.zeros(8), atol=1e-6), "Mean should be close to 0"
assert torch.allclose(stds, torch.ones(8), atol=1e-1), "Std should be close to 1"

✓ Challenge 3 complete!
Mean per feature (should be ~0): tensor([-3.8147e-08,  1.1921e-07, -3.4094e-07, -9.0599e-08, -5.5432e-08,
        -4.2915e-08, -1.6689e-08, -8.3447e-08])
Std per feature (should be ~1): tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


## Challenge 4: Batch Operations (5 minutes)

**Task:** Split 1000 patients into batches of 32 using PyTorch's DataLoader.

**Requirements:**
1. Create synthetic data for 1000 patients with 8 features
2. Create corresponding labels (0 or 1)
3. Create a `TensorDataset`
4. Create a `DataLoader` with batch_size=32, shuffle=True

In [12]:
from torch.utils.data import TensorDataset, DataLoader

# TODO: Create data for 1000 patients
X = torch.randn(1000, 8)  # Shape: (1000, 8)
y = torch.randint(0, 2, (1000,)) # Shape: (1000,) - 0 or 1 labels

# TODO: Create TensorDataset
dataset = TensorDataset(X, y)  # TensorDataset(X, y)

# TODO: Create DataLoader with batch_size=32
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Verification (don't modify)
assert X is not None and y is not None, "Create X and y!"
assert X.shape == (1000, 8), f"Expected X shape (1000, 8), got {X.shape}"
assert y.shape == (1000,), f"Expected y shape (1000,), got {y.shape}"
assert dataloader is not None, "Create the dataloader!"

# Test the dataloader
first_batch = next(iter(dataloader))
print("✓ Challenge 4 complete!")
print(f"Number of batches: {len(dataloader)}")
print(f"First batch X shape: {first_batch[0].shape}")
print(f"First batch y shape: {first_batch[1].shape}")

✓ Challenge 4 complete!
Number of batches: 32
First batch X shape: torch.Size([32, 8])
First batch y shape: torch.Size([32])


## Challenge 5: GPU Transfer (5 minutes)

**Task:** Move tensors between CPU and GPU (if available).

**Requirements:**
1. Check if CUDA is available
2. Create a device variable (cuda or cpu)
3. Move a tensor to the device
4. Perform an operation on the device
5. Move the result back to CPU

In [13]:
# TODO: Set device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Create a sample tensor
sample_tensor = torch.randn(100, 8)

# TODO: Move tensor to device
tensor_on_device = sample_tensor.to(device)


# TODO: Perform operation on device (e.g., multiply by 2)
result_on_device = tensor_on_device * 2


# TODO: Move result back to CPU
result_cpu = result_on_device.cpu()



# Verification (don't modify)
assert device is not None, "Set the device!"
assert tensor_on_device is not None, "Move tensor to device!"
assert result_cpu is not None, "Move result to CPU!"
print("✓ Challenge 5 complete!")
print(f"Device: {device}")
print(f"Tensor on device: {tensor_on_device.device}")
print(f"Result on CPU: {result_cpu.device}")

✓ Challenge 5 complete!
Device: cuda
Tensor on device: cuda:0
Result on CPU: cpu


---

# 🧠 PART 2: Build Your First Neural Network (60 minutes)

## The Challenge

Build a **neural network to diagnose diabetes** from patient medical data.

**Dataset:** Pima Indians Diabetes Dataset (familiar from Lab 1!)
- 768 patients
- 8 features: pregnancies, glucose, blood pressure, skin thickness, insulin, BMI, diabetes pedigree, age
- Target: Diabetes diagnosis (0 = no, 1 = yes)

**Your Task:**
1. Load and prepare the data
2. Define a neural network architecture
3. **Implement the training loop from scratch** ← This is the most important part!
4. Evaluate with medical metrics
5. Experiment and optimize

---

## 📊 Milestone 1: Data Preparation (10 minutes)

**Objectives:**
1. Load the Pima diabetes dataset
2. Split into features (X) and labels (y)
3. Convert to PyTorch tensors
4. Split into train/validation/test sets (60/20/20)
5. Create DataLoaders with batch_size=32

**Hints provided, but YOU write the code!**

In [ ]:
# Download the dataset
!wget -q https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv -O diabetes.csv

In [14]:
# Import additional libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: Load the data
# Hint: df = pd.read_csv('diabetes.csv', header=None)
#       Column names: ['Pregnancies','Glucose','BloodPressure','SkinThickness',
#                      'Insulin','BMI','DiabetesPedigree','Age','Outcome']
df = pd.read_csv('diabetes.csv', header=None)


# TODO: Separate features (X) and labels (y)
# Hint: X = df.iloc[:, :-1].values  # All columns except last
#       y = df.iloc[:, -1].values   # Last column only
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values



# TODO: Split into train/val/test (60/20/20)
# Hint: First split into train (60%) and temp (40%)
#       Then split temp into val (50% of temp = 20% overall) and test (50% of temp = 20% overall)
#       Use stratify=y to maintain class balance
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)




# TODO: Normalize the features (using training set statistics)
# Hint: scaler = StandardScaler()
#       X_train_scaled = scaler.fit_transform(X_train)
#       X_val_scaled = scaler.transform(X_val)
#       X_test_scaled = scaler.transform(X_test)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)



# TODO: Convert to PyTorch tensors
# Hint: X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
#       y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)




# TODO: Create TensorDatasets
# Hint: train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)



# TODO: Create DataLoaders with batch_size=32
# Hint: train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # No need to shuffle validation data
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # No need to shuffle test data



# ✓ CHECKPOINT: Verify shapes
print("Data Preparation Complete!")
print(f"Train set: {X_train_tensor.shape[0]} samples")
print(f"Val set: {X_val_tensor.shape[0]} samples")
print(f"Test set: {X_test_tensor.shape[0]} samples")
print(f"Features: {X_train_tensor.shape[1]}")
print(f"Batches in train_loader: {len(train_loader)}")

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

## 🏗️ Milestone 2: Define the Neural Network (15 minutes)

**Architecture:**
```
Input Layer:    8 features
Hidden Layer 1: 16 neurons + ReLU activation
Hidden Layer 2: 8 neurons + ReLU activation
Output Layer:   2 neurons (no diabetes / diabetes)
```

**Your Task:** Build this network using `nn.Module`

**Key Concepts:**
- `nn.Linear(in_features, out_features)` - Fully connected layer
- `nn.ReLU()` - Activation function
- `forward()` - Define how data flows through the network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# TODO: Define the neural network class
class DiabetesClassifier(nn.Module):
    def __init__(self):
        super(DiabetesClassifier, self).__init__()

        # TODO: Define layers
        # Hint: self.fc1 = nn.Linear(8, 16)  # Input to hidden1
        #       self.relu = nn.ReLU()
        #       self.fc2 = nn.Linear(16, 8)  # Hidden1 to hidden2
        #       self.fc3 = nn.Linear(8, 2)   # Hidden2 to output




    def forward(self, x):
        # TODO: Implement forward pass
        # Hint: x = self.fc1(x)
        #       x = self.relu(x)
        #       x = self.fc2(x)
        #       x = self.relu(x)
        #       x = self.fc3(x)
        #       return x




        pass

# TODO: Instantiate the model
model = None  # DiabetesClassifier()



# TODO: Print model architecture
# print(model)


# ✓ CHECKPOINT: Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

if model is not None:
    print(f"\nTotal parameters: {count_parameters(model)}")
    print("Expected: ~300 parameters")

    # Test forward pass
    test_input = torch.randn(1, 8)
    test_output = model(test_input)
    print(f"\nTest input shape: {test_input.shape}")
    print(f"Test output shape: {test_output.shape}")
    print("Expected output shape: torch.Size([1, 2])")

## ⚙️ Milestone 3: Training Setup (10 minutes)

**Components needed:**
1. Loss function - `CrossEntropyLoss` (for classification)
2. Optimizer - `Adam` (adaptive learning rate)
3. Learning rate - Start with 0.001
4. Device - GPU if available, CPU otherwise
5. Number of epochs - 50

**Why these choices?**
- CrossEntropyLoss: Standard for multi-class classification
- Adam: Generally works well, adapts learning rate automatically
- lr=0.001: Good starting point, can tune later
- 50 epochs: Enough to see convergence, not too slow

In [ ]:
import torch.optim as optim

# TODO: Define loss function
# Hint: criterion = nn.CrossEntropyLoss()
criterion = None


# TODO: Define optimizer
# Hint: optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = None


# TODO: Set device (GPU if available)
# Hint: device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = None


# TODO: Move model to device
# Hint: model = model.to(device)


# Set number of epochs
num_epochs = 50

# ✓ CHECKPOINT: Verify setup
print("Training Setup Complete!")
print(f"Device: {device}")
print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
print(f"Number of epochs: {num_epochs}")

## 🔄 Milestone 4: Training Loop - THE MOST IMPORTANT! (15 minutes)

**This is where the learning happens!**

For each epoch:
1. **Training Phase:**
   - Set model to training mode
   - For each batch:
     - Move batch to device
     - Zero gradients (IMPORTANT!)
     - Forward pass
     - Compute loss
     - Backward pass (compute gradients)
     - Update weights

2. **Validation Phase:**
   - Set model to evaluation mode
   - No gradient computation
   - Compute validation loss and accuracy

**This is the CORE of deep learning - understand every line!**

In [ ]:
# Lists to store metrics
train_losses = []
val_losses = []
val_accuracies = []

print("Starting training...\n")

for epoch in range(num_epochs):
    # ============================================
    # TRAINING PHASE
    # ============================================

    # TODO: Set model to training mode
    # Hint: model.train()


    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        # TODO: Move batch to device
        # Hint: batch_X = batch_X.to(device)
        #       batch_y = batch_y.to(device)



        # TODO: Zero the gradients (CRITICAL!)
        # Hint: optimizer.zero_grad()
        # Why? Gradients accumulate by default - we need to clear them each iteration



        # TODO: Forward pass
        # Hint: outputs = model(batch_X)



        # TODO: Compute loss
        # Hint: loss = criterion(outputs, batch_y)



        # TODO: Backward pass (compute gradients)
        # Hint: loss.backward()
        # This is backpropagation!



        # TODO: Update weights
        # Hint: optimizer.step()



        # Accumulate loss
        train_loss += loss.item()

    # TODO: Calculate average training loss
    # Hint: train_loss = train_loss / len(train_loader)



    # ============================================
    # VALIDATION PHASE
    # ============================================

    # TODO: Set model to evaluation mode
    # Hint: model.eval()


    val_loss = 0.0
    correct = 0
    total = 0

    # TODO: Disable gradient computation (saves memory and computation)
    # Hint: with torch.no_grad():

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            # TODO: Move batch to device



            # TODO: Forward pass



            # TODO: Compute loss



            # Accumulate loss
            val_loss += loss.item()

            # TODO: Compute accuracy
            # Hint: _, predicted = torch.max(outputs, 1)
            #       total += batch_y.size(0)
            #       correct += (predicted == batch_y).sum().item()





    # TODO: Calculate average validation loss and accuracy
    # Hint: val_loss = val_loss / len(val_loader)
    #       val_accuracy = 100 * correct / total



    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Print progress every 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val Accuracy: {val_accuracy:.2f}%\n")

print("✓ Training Complete!")

### 📈 Visualize Training Progress

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax1.plot(train_losses, label='Training Loss', linewidth=2)
ax1.plot(val_losses, label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy curve
ax2.plot(val_accuracies, label='Validation Accuracy', color='green', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accuracies[-1]:.2f}%")

## 📊 Milestone 5: Medical Evaluation Metrics (10 minutes)

**In medical diagnosis, accuracy is NOT enough!**

We need:
- **Sensitivity (Recall)** - How many actual diabetics did we catch?
- **Specificity** - How many non-diabetics did we correctly identify?
- **Precision** - Of those we diagnosed as diabetic, how many actually are?
- **F1-Score** - Harmonic mean of precision and recall

**Why this matters:**
- Missing a diabetic patient (False Negative) can be life-threatening
- Falsely diagnosing diabetes (False Positive) causes unnecessary treatment
- We need to balance these based on clinical priorities

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# TODO: Set model to evaluation mode
# Hint: model.eval()


# Get predictions on test set
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        # TODO: Move to device


        # TODO: Get predictions
        # Hint: outputs = model(batch_X)
        #       _, predicted = torch.max(outputs, 1)



        # Store predictions and labels
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Convert to numpy arrays
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# TODO: Compute metrics
# Hint: accuracy = accuracy_score(all_labels, all_predictions)
#       precision = precision_score(all_labels, all_predictions)
#       sensitivity = recall_score(all_labels, all_predictions)  # Sensitivity = Recall
#       f1 = f1_score(all_labels, all_predictions)

accuracy = None
precision = None
sensitivity = None  # This is Recall
f1 = None




# TODO: Compute specificity manually
# Hint: cm = confusion_matrix(all_labels, all_predictions)
#       tn, fp, fn, tp = cm.ravel()
#       specificity = tn / (tn + fp)

cm = None
specificity = None




# Print results
print("\n" + "="*50)
print("MEDICAL EVALUATION METRICS")
print("="*50)
print(f"Accuracy:    {accuracy*100:.2f}%")
print(f"Precision:   {precision*100:.2f}%  (Of predicted diabetic, how many are correct?)")
print(f"Sensitivity: {sensitivity*100:.2f}%  (Of actual diabetic, how many did we catch?)")
print(f"Specificity: {specificity*100:.2f}%  (Of actual non-diabetic, how many correct?)")
print(f"F1-Score:    {f1:.4f}")
print("="*50)

# Baseline comparison
baseline_accuracy = max(np.mean(all_labels), 1 - np.mean(all_labels))
print(f"\nBaseline (always predict majority class): {baseline_accuracy*100:.2f}%")
print(f"Our model improvement: +{(accuracy - baseline_accuracy)*100:.2f}%")

### 🎯 Confusion Matrix Visualization

In [ ]:
# TODO: Create confusion matrix heatmap
# Hint: Use seaborn heatmap

plt.figure(figsize=(8, 6))

# Create heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Diabetes', 'Diabetes'],
            yticklabels=['No Diabetes', 'Diabetes'])

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix\n')

# Add interpretation
tn, fp, fn, tp = cm.ravel()
plt.text(0.5, -0.15, f"True Negatives: {tn}", transform=plt.gca().transAxes, ha='center')
plt.text(0.5, -0.20, f"False Positives: {fp} (unnecessary treatment)", transform=plt.gca().transAxes, ha='center')
plt.text(0.5, -0.25, f"False Negatives: {fn} (missed diagnoses - CRITICAL!)", transform=plt.gca().transAxes, ha='center', color='red')
plt.text(0.5, -0.30, f"True Positives: {tp}", transform=plt.gca().transAxes, ha='center')

plt.tight_layout()
plt.show()

print(f"\nClinical Interpretation:")
print(f"- We correctly identified {tp} diabetic patients")
print(f"- We MISSED {fn} diabetic patients (this is bad!)")
print(f"- We falsely diagnosed {fp} healthy patients (unnecessary worry/treatment)")
print(f"- We correctly identified {tn} healthy patients")

### 📋 Detailed Classification Report

In [ ]:
# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_predictions,
                          target_names=['No Diabetes', 'Diabetes']))

## 🔬 Milestone 6: Experimentation & Optimization (Remaining time)

**Now it's YOUR turn to improve the model!**

Try different approaches and document what works:

### Ideas to Experiment With:

1. **Architecture Changes:**
   - More/fewer layers
   - Different layer sizes
   - Add dropout for regularization
   - Add batch normalization

2. **Training Changes:**
   - Different learning rates (0.0001, 0.01)
   - Different optimizers (SGD, RMSprop)
   - Different batch sizes (16, 64)
   - More/fewer epochs

3. **Data Changes:**
   - Different train/val/test splits
   - Different normalization
   - Handle class imbalance (weighted loss)

**Keep track of your experiments!**

### Experiment 1: Deeper Network

In [ ]:
# TODO: Try a deeper network
# Example: 8 → 32 → 16 → 8 → 2

class DeeperClassifier(nn.Module):
    def __init__(self):
        super(DeeperClassifier, self).__init__()
        # Your architecture here
        pass

    def forward(self, x):
        # Your forward pass here
        pass

# Train and evaluate
# (Copy training loop from above and modify)

# Document results:
# Accuracy: _____
# Sensitivity: _____
# Better than original? _____

### Experiment 2: Add Dropout

In [ ]:
# TODO: Add dropout layers (e.g., 0.2 or 0.5)
# Dropout helps prevent overfitting

class DropoutClassifier(nn.Module):
    def __init__(self):
        super(DropoutClassifier, self).__init__()
        # Add dropout: self.dropout = nn.Dropout(0.2)
        pass

    def forward(self, x):
        # Apply dropout between layers
        pass

# Document results:
# Accuracy: _____
# Did it reduce overfitting? _____

### Experiment 3: Different Learning Rate

In [ ]:
# TODO: Try lr=0.0001 (lower) or lr=0.01 (higher)

# optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Document results:
# Learning rate: _____
# Convergence speed: _____
# Final accuracy: _____

### Experiment 4: Class Imbalance Handling

In [ ]:
# TODO: Use weighted loss to handle class imbalance
# Medical datasets often have imbalanced classes

# Calculate class weights
# class_counts = np.bincount(y_train)
# class_weights = 1. / class_counts
# weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
# criterion = nn.CrossEntropyLoss(weight=weights)

# Document results:
# Did sensitivity improve? _____

### 📊 Experiment Comparison Table

In [ ]:
# TODO: Create a comparison table of all your experiments

results = {
    'Experiment': ['Baseline', 'Deeper', 'Dropout', 'Low LR', 'Weighted Loss'],
    'Accuracy': [None, None, None, None, None],
    'Sensitivity': [None, None, None, None, None],
    'Specificity': [None, None, None, None, None],
    'F1-Score': [None, None, None, None, None]
}

results_df = pd.DataFrame(results)
print(results_df)

# Which experiment performed best?
# Your analysis:

---

## 🎓 Reflection Questions

Answer these questions based on your experiments:

1. **What was the biggest challenge in building the neural network?**
   - Your answer:

2. **Which component of the training loop did you find most confusing initially?**
   - Your answer:

3. **Why is sensitivity (recall) particularly important for medical diagnosis?**
   - Your answer:

4. **What was your best performing model configuration?**
   - Your answer:

5. **If you were deploying this in a real hospital, what additional considerations would you have?**
   - Your answer:

---

## 🎉 Lab 2 Complete!

### What You Accomplished:

✅ Mastered PyTorch tensor operations  
✅ Built a complete neural network from scratch  
✅ Implemented the full training loop  
✅ Evaluated with medical metrics  
✅ Experimented with improvements  

### Key Takeaways:

- **The training loop is fundamental** - forward, loss, backward, optimize
- **Medical metrics matter** - accuracy alone is insufficient
- **Experimentation is essential** - no "perfect" hyperparameters
- **Neural networks are powerful** but require careful tuning

**Good work! 🎓✨**